<h1 align="center">Python performance exercises</h1>

## Python best practices exercises

### Exercise 1

considering the following function for concatenating list strings with delimiter.

In [1]:
def ft_concatenate(l_strings, d):
    """concatenate list of strings into one string separated by delimiter"""
    res = l_strings[0]
    for e in l_strings[1:]:
        res = res + d + e
    return res

- profile the function and identify the bottlenecks.
- improve speed up of the function
*Hint: you may need to look to the string functions in python documentation*

In [2]:
# write your code here

%timeit ft_concatenate([str(i) for i in range(100000)],'$')

1.41 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
L=[str(i) for i in range(100000)]
%prun  ft_concatenate(L,'$')

In [5]:
def ft_concatenate_opt(l_strings, d):
    """concatenate list of strings into one string separated by delimiter"""
    res = d.join(l_strings)
    return res

In [6]:
%timeit ft_concatenate_opt(L,'$')

1.17 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%prun  ft_concatenate_opt(L,'$')

### Exercise 2

In this exercise you will solve the following problem using two methods bruteforce method, and fast method.

**Problem:** You are given a list of n integers, and your task is to calculate the number of distinct values in the list.

**Example**
- Input:
5
2 3 2 2 3

- Output:
2

**Implement the following methods:**

1. **bruteforce method:** create an empty list and start adding items for the given list without adding the previous item add, at the end the result list will contain unique values, print lenght of the list and you are done. 
2. **fast method** think of using Set data structure.

- time the two methods, what do you think?

In [4]:
# bruteforce method
def dist_val(L):
    D = []
    for l in L:
        if l not in D:
            D.append(l)
    return len(D)

In [7]:
# fast method
def distinctVal(L):
    res=set(L)
    return len(res)

In [9]:
import random
# Create a random list of numbers for testing
L = [random.randint(1, 10) for i in range(1000)]
# time the two methods
%timeit dist_val(L)
%timeit distinctVal(L)




77.2 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
7.3 µs ± 367 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Cython exercises

### Exercise 1

1. load the cython extension.

In [1]:
%load_ext cython


2. Considering the following polynomial function:

In [2]:
def poly(a,b):
    return 10.5 * a + 3 * (b**2)

- Create an equivalent Cython function of `poly` with name `poly_cy`.

In [3]:
%%cython -a
import cython
cimport cython
def poly_cy(double a, double b):
    
    return (10.5 * a) + (3 * (b**2))

3. time the performance of Python and Cython version of the function, what is the factor of speed up between the two verions.

In [4]:
%timeit poly(10**200,10**100)
%timeit poly_cy(10**200,10**200)

1.18 µs ± 2.25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
939 ns ± 0.127 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


4. Now let's work on another example using loop.
    - rewrite the same function below fib that calculates the fibonacci sequence using cython, but now try to add type for the variables used inside it, add a prefix `_cy` to your new cython function.

In [6]:
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a

    return a

In [5]:
%%cython -a

cpdef int fib_cy(int n):
    cdef int i,a,b
    a, b = 1, 1
    for i in range(n-1):
        a, b = a + b, a

    return a

- time the two function for fibonacci series, with n = 20, what is the factor of speed now, What do you think?

In [7]:
%timeit fib(20)
%timeit fib_cy(20)

811 ns ± 1.71 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
41.3 ns ± 1.62 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


5. Recursive functions are functions that call themselves during their execution. Another interesting property of the Fibonacci sequence is that it can be written as a recursive function. That’s because each item depends on the values of other items (namely item n-1 and item n-2)

- Rewrite the fib function using recursion. Is it faster than the non-recursive version? Does Cythonizing it give even more of an advantage? 

In [8]:
def fib_reccur(n):
    if n==0 or n==1:
        return 1
    if n==2:
        return 2
    return fib_reccur(n-1)+fib_reccur(n-2)

In [9]:
%%cython -a

cpdef int fib_reccur_cy(int n):
    if n==0 or n==1:
        return 1
    if n==2:
        return 2
    return fib_reccur_cy(n-1)+fib_reccur_cy(n-2)

In [10]:
%timeit fib_reccur(20)
%timeit fib_reccur_cy(20)

1.17 ms ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.7 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


We see that the the non-recursive method is much faster than the recursive methode ,but when we cythonize both of them they get faster than before.

### Exercise 2

- Monte Carlo methods are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. 
- One of the basic examples of getting started with the Monte Carlo algorithm is the estimation of Pi.

**Estimation of Pi**

- The idea is to simulate random (x, y) points in a 2-D plane with domain as a square of side 1 unit. 
- Imagine a circle inside the same domain with same diameter and inscribed into the square. 
- We then calculate the ratio of number points that lied inside the circle and total number of generated points. 
- Refer to the image below:

![demo](../data/MonteCarloPlot.png)

We know that area of the square is 1 unit sq while that of circle is $\pi \ast  (\frac{1}{2})^{2} = \frac{\pi}{4}$. Now for a very large number of generated points,

![demo](../data/MonteCarloCalc.png)


## The Algorithm

1. Initialize cile_points, square_points and interval to 0.
2. Generate random point x.
3. Generate random point y.
4. Calculate d = x*x + y*y.
5. If d <= 1, increment circle_points.
6. Increment square_points.
7. Increment interval.
8. If increment < NO_OF_ITERATIONS, repeat from 2.
9. Calculate pi = 4*(circle_points/square_points).
10. Terminate.

**Your mission:** time the function `monte_carlo_pi`, identify the bottlenecks and create a new version using cython functionality to speed up monte carlo simulation for PI, use 100,000 points and compare the speed up factor between python and cython, considering the following optimizations:
- add type for variables used.
- add type for the function
- use c rand function instead of python rand function.
 
*Hint: you can import function from C libraries using the following approach `from libc.<name of c library> cimport <library function name>`, replace the holders `<>` with the right identities for the current problem*

In [18]:
import random
def monte_carlo_pi(nsamples):
    pi = 0.
    cile_points= 0
    square_points= 0
    interval=0
    for i in range(nsamples):
        x=random.uniform(-1, 1)
        y=random.uniform(-1, 1)
        d=x**2 + y**2 # square of distance between M(x,y) and o(0,0)
        if d<= 1: # if the point M(x,y) is inside the circle
            cile_points+= 1
        square_points+= 1
        interval+=1

    pi=4*(cile_points/square_points)
    
    return pi

In [26]:
monte_carlo_pi(100000000)

3.14163928

### speed up the function

In [27]:
%%cython 
from libc.stdlib cimport rand, srand, RAND_MAX
from libc.time cimport time
cpdef float monte_carlo_pi_cy(int nsamples):
    cdef int i,cile_points=0
    cdef float x,y,pi
    srand(time(NULL))
    for i in range(nsamples):
        x=rand()/RAND_MAX
        y=rand()/RAND_MAX
        d=x**2 + y**2 # square of distance between M(x,y) and o(0,0)
        if d<= 1: # if the point M(x,y) is inside the circle
            cile_points+= 1
    #print((cile_points))        
    pi=4*(cile_points/nsamples)
    return pi

In [29]:
  monte_carlo_pi_cy(100000000)

3.14141845703125

### Comparing the speed up factor between the two versions

In [30]:
%timeit monte_carlo_pi(10000000)
%timeit monte_carlo_pi_cy(10000000)

6.01 s ± 5.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
486 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numba exercises

### Exercise 1

Previously we considered how to approximateby Monte Carlo.

- Use the same idea here, but make the code efficient using Numba.
- Compare speed with and without Numba when the sample size is large.

In [38]:
from numba import njit
import random

In [41]:
@njit
def monte_carlo_pi_numba(nsamples):
    pi = 0.
    cile_points= 0
    square_points= 0
    interval=0
    for i in range(nsamples):
        x=random.uniform(-1, 1)
        y=random.uniform(-1, 1)
        d=x**2 + y**2 # square of distance between M(x,y) and o(0,0)
        if d<= 1: # if the point M(x,y) is inside the circle
            cile_points+= 1
        square_points+= 1
        interval+=1

    pi=4*(cile_points/square_points)
    
    return pi

In [42]:
%timeit monte_carlo_pi_numba(10000000)

92.7 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Exercise 2

In the [Introduction to Quantitative Economics](https://python.quantecon.org/intro.html) with Python lecture series you can learn all about finite-state Markov chains.

For now, let's just concentrate on simulating a very simple example of such a chain.

Suppose that the volatility of returns on an asset can be in one of two regimes — high or low.

The transition probabilities across states are as follows ![markov](../data/markov.png)

For example, let the period length be one day, and suppose the current state is high.

We see from the graph that the state tomorrow will be

- high with probability 0.8

- low with probability 0.2

Your task is to simulate a sequence of daily volatility states according to this rule.

Set the length of the sequence to `n = 1_000_000` and start in the high state.

Implement a pure Python version and a Numba version, and compare speeds.

To test your code, evaluate the fraction of time that the chain spends in the low state.

If your code is correct, it should be about 2/3.

Hints:

- Represent the low state as 0 and the high state as 1.

- If you want to store integers in a NumPy array and then apply JIT compilation, use `x = np.empty(n, dtype=np.int_)`.


In [58]:
import numpy as np
@njit
def bern(p):
    u = np.random.uniform(0,1)
    if(u<p):
        return 1
    return 0

In [73]:
def markov(nsamples):
    state = [1]
    for i in range(nsamples):
        if(state[i]==1):
            if(bern(0.8)):
                state.append(1)
            else:
                state.append(0)
        if(state[i]==0):
            if(bern(0.9)):
                state.append(0)
            else:
                state.append(1)
    return state.count(0)/nsamples

In [74]:
markov(10000000)

0.6670409

In [75]:
@njit
def markov2(nsamples):
    state = np.empty(nsamples, dtype=np.int32)
    state[0] = 1
    for i in range(nsamples-1):
        if(state[i]==1):
            if(bern(0.8)):
                state[i+1] = 1
            else:
                state[i+1] = 0
        if(state[i]==0):
            if(bern(0.9)):
                state[i+1] = 0
            else:
                state[i+1] = 1
    print
    c = nsamples - np.count_nonzero(state)
    return c/nsamples

In [76]:
markov2(1000000)

0.666742

In [77]:
%timeit markov(1000000)
%timeit markov2(1000000)

256 ms ± 3.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
25.8 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
